# Lee un año de datos a ver cómo se hace más rápido - Versión lee todos los meses de golpe

In [1]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/09 17:41:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Wed Nov  9 17:41:51 2022


In [3]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaCreacion",StringType(),True)

def listPartitions (anoMesFinal, meses, path):
    lista = []
    mes = int(anoMesFinal[-2:]) + 1
    ano = int(anoMesFinal[:-3])
    
    print('Primer mes: '+anoMesFinal)
    
    for i in range(meses):
        mes = mes - 1
        if mes == 0:
            ano = ano - 1
            mes = 12
        if mes < 10: anoMes = str(ano)+'-0'+str(mes)
        else: anoMes = str(ano)+'-'+str(mes)

        lista.append(path+anoMes)
    print('Último mes: '+anoMes)
    return lista

df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fakePeopleConPartido.parquet/")\
               .format("parquet").load(listPartitions('1932-12', 2*12, './datos/fakePeopleConPartido.parquet/yearMon='))

ct = df.count()
print(ct)

Primer mes: 1932-12
Último mes: 1931-01


19281721
CPU times: user 7.54 ms, sys: 4.65 ms, total: 12.2 ms
Wall time: 5.73 s


In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

Final: Wed Nov  9 17:41:56 2022
Hecho en 5.7713 segundo(s)


In [5]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
